# ML-5333 Project 2 - MLB Data Collection

In [3]:
import pandas as pd
import numpy as np
import baseball_scraper
import pybaseball
import statsapi
import mlbgame

In [11]:
with open('data\\team_names.txt', 'r') as file:
    team_list = [x.replace('\n','') for x in file.readlines()]
team_df = []
for team in team_list:
    team_df.append(statsapi.lookup_team(team)[0])
team_df = pd.DataFrame.from_dict(team_df)
team_df.columns = ['statsapi_id','name','team_code','file_code','team_name','loc_name','short_name']

In [13]:
team_df = team_df.sort_values(by=['team_name'])
team_df

,statsapi_id,name,team_code,file_code,team_name,loc_name,short_name
26,108,Los Angeles Angels,ana,ana,Angels,Anaheim,LA Angels
25,117,Houston Astros,hou,hou,Astros,Houston,Houston
27,133,Oakland Athletics,oak,oak,Athletics,Oakland,Oakland
19,141,Toronto Blue Jays,tor,tor,Blue Jays,Toronto,Toronto
0,144,Atlanta Braves,atl,atl,Braves,Atlanta,Atlanta
7,158,Milwaukee Brewers,mil,mil,Brewers,Milwaukee,Milwaukee
9,138,St. Louis Cardinals,sln,stl,Cardinals,St. Louis,St. Louis
5,112,Chicago Cubs,chn,chc,Cubs,Chicago,Chi Cubs
10,109,Arizona Diamondbacks,ari,ari,D-backs,Phoenix,Arizona
12,119,Los Angeles Dodgers,lan,la,Dodgers,Los Angeles,LA Dodgers
